# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

EL OBJETIVO DE LA LIBRERIA SQLALCHEMY ES HACER DE TRADUCTOR DE SINTAXIS ENTRE SERVIDORES SQL,SUS COMANDOS Y CODIGO PYHTON.
PRIMERO DE TODO INSTALAMOS LAS DEPENDENCIAS MEDIANTE TERMINAL GIT CON PIP.
pip install sqlalchemy python-dotenv psycopg2-binary.
SI hay errores de incompatibilidad tendremos que utilizar diferentes versiones de python hasta encontrar una que no haya errores de compatibilidad entre las dependencias de cada libreria.
UTILIZARE MI PROPIO SERVIDOR POSTGRESQL INSTALADO EN MI SOBREMESA Y PREVIAMENTE CONECTADO MEDIANTE SSL, PREVIAMENTE CONFIGURADO CON PGADMIN4.
LAS CREDENCIALES SE ENLAZARAN A LA LLAMADA DE UNA VARIABLE EXTERNA A ESTE DOCUMENTO, EL MISMO SERA INVISIBLE PARA LOS USUARIOS DE ESTE WORKSPACE EN REMOTO.
PARA ELLO SE ANOTARA EL NOMBRE DEL DOCUMENTO EN EL .GITIGNORE, EL ARCHIVO EN QUESTION ES UN .ENV QUE SE UTILIZA BASICAMENTE COMO ARCHIVO DE CONFIGURACION.
PARA OPERAR CON ESTE TIPO DE ARCHIVO SE UTILIZA LA LIBRERIA DOTENV COMUNMENTE.

In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Date, DECIMAL, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from pathlib import Path

# Obtener la ruta absoluta del directorio actual (para Jupyter)
current_dir = Path(os.getcwd())

# Construir la ruta absoluta al archivo user.env
dotenv_path = current_dir / 'user.env'

# Cargar variables de entorno desde la ruta absoluta
load_dotenv(dotenv_path=dotenv_path)

# Configuración de la conexión a PostgreSQL
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Base = declarative_base()

# Crear una sesión
Session = sessionmaker(bind=engine)
session = Session()

# Definición del modelo de la base de datos
class Publisher(Base):
    __tablename__ = 'publishers'
    publisher_id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)

class Author(Base):
    __tablename__ = 'authors'
    author_id = Column(Integer, primary_key=True)
    first_name = Column(String(100), nullable=False)
    middle_name = Column(String(50))
    last_name = Column(String(100))

class Book(Base):
    __tablename__ = 'books'
    book_id = Column(Integer, primary_key=True)
    title = Column(String(255), nullable=False)
    total_pages = Column(Integer)
    rating = Column(DECIMAL(4, 2))
    isbn = Column(String(13))
    published_date = Column(Date)
    publisher_id = Column(Integer, ForeignKey('publishers.publisher_id'))
    publisher = relationship("Publisher")

class BookAuthor(Base):
    __tablename__ = 'book_authors'
    book_id = Column(Integer, ForeignKey('books.book_id'), primary_key=True)
    author_id = Column(Integer, ForeignKey('authors.author_id'), primary_key=True)
    book = relationship("Book")
    author = relationship("Author")

# Crear las tablas
Base.metadata.create_all(engine)

# Insertar datos (verificación de tabla vacía)
if not session.query(Publisher).first():
    publishers_data = [
        # ... (datos de publishers) ...
    ]
    session.add_all(publishers_data)

if not session.query(Author).first():
    authors_data = [
        # ... (datos de authors) ...
    ]
    session.add_all(authors_data)

if not session.query(Book).first():
    books_data = [
        # ... (datos de books) ...
    ]
    session.add_all(books_data)

if not session.query(BookAuthor).first():
    book_authors_data = [
        # ... (datos de book_authors) ...
    ]
    session.add_all(book_authors_data)

session.commit()